In [15]:
import pandas as pd

In [16]:
df_url = "https://raw.githubusercontent.com/EZags/Milliman-Project-2025/main/merged_df1.csv"
df = pd.read_csv(df_url)
#Drop non-numeric columns
df

,county_state,fips,population,dep_prevalence,Households Total,Households Mean income (dollars),Households Median income (dollars),Premature Death Deaths,Premature Death Years of Potential Life Lost Rate,Poor Physical Health Days Average Number of Physically Unhealthy Days,...,Unemployment % Unemployed,Income Inequality 80th Percentile Income,Income Inequality 20th Percentile Income,Income Inequality Income Ratio,Children in Poverty % Children in Poverty,Injury Deaths # Injury Deaths,Injury Deaths Injury Death Rate,Social Associations # Associations,Social Associations Social Association Rate,Child Care Cost Burden % Household Income Required for Child Care Expenses
0,"AUTAUGA COUNTY, ALABAMA",1001,60436,18,NaN,NaN,NaN,1008.0,9938.263382,4.300364,...,2.188889,132562.0,31018.0,4.273712,17.0,189.0,65.975725,67.0,11.211700,21.835108
1,"BALDWIN COUNTY, ALABAMA",1003,254107,18,105698.0,98279.0,72915.0,3944.0,8957.112686,3.876436,...,2.279497,141920.0,32554.0,4.359526,14.1,953.0,82.420053,239.0,9.698298,19.354057
2,"BARBOUR COUNTY, ALABAMA",1005,24644,19,NaN,NaN,NaN,587.0,12738.656137,5.431030,...,4.431017,99138.0,17658.0,5.614339,34.8,106.0,85.603993,22.0,8.904720,28.579628
3,"BIBB COUNTY, ALABAMA",1007,21890,24,NaN,NaN,NaN,509.0,11708.948038,5.055619,...,2.462380,102593.0,19188.0,5.346727,21.4,111.0,99.630201,19.0,8.634401,26.582776
4,"BLOUNT COUNTY, ALABAMA",1009,59777,22,NaN,NaN,NaN,1273.0,11898.088409,4.802400,...,2.144295,117729.0,25484.0,4.619722,16.6,287.0,98.254695,44.0,7.393467,25.313260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3236,"VEGA BAJA MUNICIPIO, PUERTO RICO",72145,0,18,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3237,"VIEQUES MUNICIPIO, PUERTO RICO",72147,0,10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3238,"VILLALBA MUNICIPIO, PUERTO RICO",72149,0,11,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3239,"YABUCOA MUNICIPIO, PUERTO RICO",72151,0,14,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Load data (assuming it's already loaded as df)
# df = pd.read_csv('your_data.csv')

# Data Cleaning
# Remove rows with missing target variable
df = df.dropna(subset=['dep_prevalence'])

# Remove columns with too many missing values (>50%)
threshold = len(df) * 0.5
df = df.dropna(thresh=threshold, axis=1)

# Remove non-numeric columns that won't be used as features
df = df.drop(['county_state', 'fips'], axis=1)

# Remove rows where population is 0 (like Puerto Rico municipalities)
df = df[df['population'] > 0]

# Define features and target
X = df.drop('dep_prevalence', axis=1)
y = df['dep_prevalence']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing pipeline
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

In [18]:
# Create pipeline with preprocessing and model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])

# Train model
pipeline.fit(X_train, y_train)

# Predictions
y_pred = pipeline.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Performance:")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Model Performance:
Mean Squared Error: 7.40
R-squared: 0.42


In [19]:
# Get feature importances
rf = pipeline.named_steps['regressor']
feature_names = numeric_features
importances = rf.feature_importances_

# Create DataFrame of feature importances
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importances = feature_importances.sort_values('importance', ascending=False)

print("\nTop 10 Important Features:")
print(feature_importances.head(10))


Top 10 Important Features:
                                              feature  importance
16  Preventable Hospital Stays Preventable Hospita...    0.069525
34     High School Completion # Completed High School    0.060995
27  Air Pollution: Particulate Matter Average Dail...    0.052953
7                       Flu Vaccinations % Vaccinated    0.044850
0                                          population    0.039469
24        Driving Alone to Work % Drive Alone to Work    0.035617
32                            Some College Population    0.034453
17      Mammography Screening % with Annual Mammogram    0.033086
1                              Premature Death Deaths    0.031307
22               Severe Housing Problems Overcrowding    0.029158


In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Assuming df is your DataFrame
# Drop rows where target is missing
df = pd.read_csv('https://raw.githubusercontent.com/EZags/Milliman-Project-2025/main/engineered_df.csv')

# Separate features and target
X = df.drop(columns=['dep_prevalence'])
y = df['dep_prevalence']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])

# Function to evaluate models
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    print(f"R² Score: {r2:.4f}")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"Root Mean Squared Error: {rmse:.4f}")
    print(f"Mean Absolute Error: {mae:.4f}")
    
    return {'R2': r2, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}

In [21]:
from sklearn.ensemble import RandomForestRegressor

rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

rf_pipeline.fit(X_train, y_train)
print("Random Forest Regressor Performance:")
rf_metrics = evaluate_model(rf_pipeline, X_test, y_test)

Random Forest Regressor Performance:
R² Score: 0.4082
Mean Squared Error: 7.5748
Root Mean Squared Error: 2.7522
Mean Absolute Error: 2.0811


In [22]:
%pip install xgboost
from xgboost import XGBRegressor

xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=100, random_state=42))
])

xgb_pipeline.fit(X_train, y_train)
print("\nXGBoost Regressor Performance:")
xgb_metrics = evaluate_model(xgb_pipeline, X_test, y_test)


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
    --------------------------------------- 2.9/150.0 MB 15.2 MB/s eta 0:00:10
   - -------------------------------------- 4.5/150.0 MB 14.9 MB/s eta 0:00:10
   - -------------------------------------- 6.8/150.0 MB 11.0 MB/s eta 0:00:13
   -- ------------------------------------- 8.7/150.0 MB 10.3 MB/s eta 0:00:14
   -- ------------------------------------- 10.2/150.0 MB 9.8 MB/s eta 0:00:15
   --- ------------------------------------ 12.1/150.0 MB 9.7 MB/s eta 0:00:15
   --- ------------------------------------ 14.2/150.0 MB 9.7 MB/s eta 0:00:15
   ---- ----------------------------------- 16.5/150.0 MB 9.9 MB/s eta 0:00:14
   ----- ---------------------------------- 19.7/150.0 MB 10.3 MB/s eta 0:00:13
   ------ --------------------------------- 22.8/150.0 MB 10.8 MB/s eta 0:00:12
   ------ --------------------------------- 25.7/150.0 MB 11.1 MB/s eta 0:00:12
   ------- -------------------------------- 28.0/150.0 MB

In [23]:
from sklearn.svm import SVR

svr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf', C=1.0, epsilon=0.1))
])

svr_pipeline.fit(X_train, y_train)
print("\nSupport Vector Regressor Performance:")
svr_metrics = evaluate_model(svr_pipeline, X_test, y_test)


Support Vector Regressor Performance:
R² Score: 0.4439
Mean Squared Error: 7.1172
Root Mean Squared Error: 2.6678
Mean Absolute Error: 1.9818


In [24]:
from sklearn.neural_network import MLPRegressor

nn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MLPRegressor(hidden_layer_sizes=(100,50),
                              activation='relu',
                              solver='adam',
                              max_iter=500,
                              random_state=42))
])

nn_pipeline.fit(X_train, y_train)
print("\nNeural Network Regressor Performance:")
nn_metrics = evaluate_model(nn_pipeline, X_test, y_test)


Neural Network Regressor Performance:
R² Score: 0.1260
Mean Squared Error: 11.1862
Root Mean Squared Error: 3.3446
Mean Absolute Error: 2.4460


In [25]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

# Define base estimators
estimators = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=100, random_state=42))
]

stacked_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', StackingRegressor(
        estimators=estimators,
        final_estimator=RidgeCV(),
        cv=5))
])

stacked_pipeline.fit(X_train, y_train)
print("\nStacked Ensemble Regressor Performance:")
stacked_metrics = evaluate_model(stacked_pipeline, X_test, y_test)


Stacked Ensemble Regressor Performance:
R² Score: 0.4174
Mean Squared Error: 7.4567
Root Mean Squared Error: 2.7307
Mean Absolute Error: 2.0630


In [26]:
metrics_df = pd.DataFrame({
    'Random Forest': rf_metrics,
    'XGBoost': xgb_metrics,
    'SVR': svr_metrics,
    'Neural Network': nn_metrics,
    'Stacked Ensemble': stacked_metrics
}).T

print("\nModel Comparison:")
print(metrics_df.sort_values(by='R2', ascending=False))


Model Comparison:
                        R2        MSE      RMSE       MAE
SVR               0.443922   7.117208  2.667810  1.981809
Stacked Ensemble  0.417394   7.456744  2.730704  2.063038
Random Forest     0.408167   7.574838  2.752242  2.081090
XGBoost           0.327700   8.604728  2.933382  2.237856
Neural Network    0.126005  11.186208  3.344579  2.445952


In [31]:
# Check the top features in the SVR model using permutation importance
from sklearn.inspection import permutation_importance

result = permutation_importance(svr_pipeline, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
importances = result.importances_mean

# Get the feature names actually used by the preprocessor
feature_names = preprocessor.transformers_[0][2]

# Ensure lengths match
min_len = min(len(feature_names), len(importances))

# Create a DataFrame for better visualization
svr_feature_importances = pd.DataFrame({
	'feature': feature_names[:min_len],
	'importance': importances[:min_len]
}).sort_values('importance', ascending=False)

print(svr_feature_importances.head(10))

                                              feature  importance
4                       Flu Vaccinations % Vaccinated    0.078511
27                       Social Associations per 1000    0.066283
7   Preventable Hospital Stays Preventable Hospita...    0.054948
2   Poor Mental Health Days Average Number of Ment...    0.048833
8       Mammography Screening % with Annual Mammogram    0.047905
6       Food Environment Index Food Environment Index    0.041600
11        Driving Alone to Work % Drive Alone to Work    0.028022
9                               Uninsured % Uninsured    0.024845
0   Poor Physical Health Days Average Number of Ph...    0.017885
17                          Unemployment % Unemployed    0.016407
